# Generate trades using dedicated service - C#

### Overview
Retrieving trades over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameter.

This sample is designed to request trades and illustrate market activity over a look back period.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** trades over the time.

### Inputs/outputs
Trades extraction sample requires instrument's identifier, date time intervals as per inputs and returns the occured trades information.

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run Trades sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"
#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

using TickTradesService = Systemathics.Apis.Services.Tick.V1.TickTradesService;
using TickTradesRequest = Systemathics.Apis.Services.Tick.V1.TickTradesRequest;
using TickTradesResponse = Systemathics.Apis.Services.Tick.V1.TickTradesResponse;
using Trade = Systemathics.Apis.Type.Shared.V1.Trade;


### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *tick trades* service, we need to specify:
* Instrument identifier : the ticker and the exchange code (MIC) define a unique instrument
* Time period selection: select start and end dates and time intervals (Using UTC Time Zone)
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
var start = new DateTime(2021, 11, 05);
var end = new DateTime(2021, 11, 05);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the tick trades request time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval()
{
 StartTime = new TimeOfDay { Hours = 12, Minutes = 00, Seconds = 00 },
 EndTime = new TimeOfDay { Hours = 20, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create requests for *trades* and instantiate the service:

In [ ]:
// Generate the tick trades request
var request = new TickTradesRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick trades service
var service = new TickTradesService.TickTradesServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve trades data

The following code snippets retrieves trades data points:

In [ ]:
// Get the trades 
// store the list of trades
var trades = new List<Tuple<Timestamp, Trade>>();

// Call the service
var call = service.TickTrades(request, headers);

// Process the responses
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    if(current.Mapping != null)
    {
        // First response contains the mapping fields
        // Skip the mapping data
        continue;
    }

    // Get the time stamp for the current trade
    var ts = current.Data.TimeStamp;

    // Trade
    if (current.Data.Trade != null) 
    {      
        trades.Add(new Tuple<Timestamp, Trade>(ts, current.Data.Trade));
    }
}

The following code snippet displays request results:

In [ ]:
// Display the trades
//--> Note : the time stamp is displayed by default (using the unix format)
//--> We can use other displayers for different output formats 
//--> For more details about the time stamp: https://developers.google.com/protocol-buffers/docs/reference/google.protobuf#google.protobuf.Timestamp 
//display(trades);

#### 4.2 Plot the trade price
Display trades data over the look back period with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.
Plot a simple line graph with the trade prices and volumes as follows: 

In [ ]:
var tradesGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = "Price",
        mode = "lines",
        x = trades.Select(elt => elt.Item1.ToDateTime()),
        y = trades.Select(elt => elt.Item2.Price)
    },
    new Scattergl
    {
        name = "Size",
        mode = "lines",
        x = trades.Select(elt => elt.Item1.ToDateTime()),
        y = trades.Select(elt => elt.Item2.Size),
        yaxis = "y2"
    }
};

var layout = new Layout.Layout{ 
    title = $"Trades for {ticker}",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Size", 
        autorange = true,
        side = "right",  
    }
};

var chart = Chart.Plot(tradesGraph, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);